In [1]:
from typing import TypedDict, Annotated
from langgraph.graph import add_messages, StateGraph, END
from langchain_groq import ChatGroq
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3

In [3]:
connection = sqlite3.connect("checkpoint.sqlite",check_same_thread=False)
memory = SqliteSaver(conn=connection)

In [4]:

groq_api = "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"

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    groq_api_key=groq_api
)


In [5]:
class BasicState(TypedDict):
    messages: Annotated[list,add_messages]

    
def llm_node(state: BasicState):
    return {
        "messages":[llm.invoke(state["messages"])]
    }


In [6]:
graph = StateGraph(BasicState)
graph.add_node("llm_node",llm_node)
graph.add_edge("llm_node",END)
graph.set_entry_point("llm_node")
app = graph.compile(checkpointer=memory)


In [7]:
config = { "configurable":{
   "thread_id":1
}
}

In [11]:
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break
    response = app.invoke({"messages":[HumanMessage(user_input)]},config=config)
    print(f"Bot: {response['messages'][-1].content}")


Bot: Your name is Vinith.
Bot: You told me you're 21 years old.
Bot: If you need anything else, feel free to ask. Have a great day!
